<a href="https://colab.research.google.com/github/Rumata-arc/Probabilities_Surprisal/blob/main/Token_word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =========================
# БЛОК 1: Импорт библиотек и настройки путей
# =========================
import os
import math
import zipfile
import pandas as pd
import openpyxl
from transformers import AutoTokenizer

ZIP_PATH = "/mnt/data/Qwen1.5_ALL-RESULTS.zip"
EXCEL_PATH = "/mnt/data/data_exp.xlsx"
OUT_DIR = "/mnt/data/wordlevel_out"
os.makedirs(OUT_DIR, exist_ok=True)

# ВАЖНО: должен совпадать с тем, что ты использовал при расчёте вероятностей.
# Если у тебя другой чекпойнт/путь — поменяй.
TOKENIZER_NAME_OR_PATH = "Qwen/Qwen1.5-0.5B"

# Режим "мягкого сдвига" при NO_MATCH:
# True  -> если слово не сматчилось, сдвигаем pos на 1 токен и идём дальше (лучше для "не падать")
# False -> если слово не сматчилось, сразу останавливаем обработку текста (лучше для строгой диагностики)
SOFT_SHIFT_ON_NO_MATCH = True


In [ ]:
# =========================
# БЛОК 2: Загружаем токенайзер (модель НЕ нужна)
# =========================
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME_OR_PATH, use_fast=True)
print("Tokenizer loaded:", TOKENIZER_NAME_OR_PATH)


In [ ]:
# =========================
# БЛОК 3: Вспомогательные функции
# - извлечение слов из листа Excel (колонки=предложения; читаем слева направо, внутри сверху вниз)
# - кодирование слова в токены (кандидаты с/без ведущего пробела)
# - матчинг кандидата с реальным потоком token_id из CSV
# =========================
def extract_words_from_sheet(ws, start_row=2, start_col=1):
    """Возвращает список слов в порядке чтения (continuous stream)."""
    words = []
    for col in range(start_col, ws.max_column + 1):
        header = ws.cell(row=1, column=col).value
        if header is None:
            continue
        for row in range(start_row, ws.max_row + 1):
            v = ws.cell(row=row, column=col).value
            if v is None:
                break
            s = str(v).strip()
            if s == "":
                break
            words.append(s)
    return words


def encode_word_candidates(word, is_first):
    """
    Возвращает несколько вариантов токенизации слова.
    Термины:
    - leading-space convention (ведущий пробел): многие BPE-токенизаторы кодируют пробел как часть следующего токена.
    """
    w = str(word)

    cands = []
    # "ожидаемое" поведение: первое слово без пробела, остальные с ведущим пробелом
    if is_first:
        cands.append(tokenizer.encode(w, add_special_tokens=False))
    else:
        cands.append(tokenizer.encode(" " + w, add_special_tokens=False))

    # альтернативы на случай отличий в сборке текста
    cands.append(tokenizer.encode(w, add_special_tokens=False))
    cands.append(tokenizer.encode("  " + w, add_special_tokens=False))  # двойной пробел (редко, но безопасно проверить)
    cands.append(tokenizer.encode("\n" + w, add_special_tokens=False))  # если где-то был перенос

    # убрать дубликаты
    uniq = []
    seen = set()
    for ids in cands:
        t = tuple(ids)
        if t not in seen and len(ids) > 0:
            seen.add(t)
            uniq.append(ids)
    return uniq


def best_match_token_ids(token_ids_stream, pos, word, is_first):
    """
    Смотрит: совпадает ли какой-то вариант токенизации слова с token_ids_stream[pos:pos+L]
    Возвращает (ids, L) или (None, 0)
    """
    for ids in encode_word_candidates(word, is_first):
        L = len(ids)
        if token_ids_stream[pos:pos+L] == ids:
            return ids, L
    return None, 0


In [ ]:
# =========================
# БЛОК 4: Преобразование token-level -> word-level (continuous stream)
# На вход: token_df (Text_X_probabilities.csv), words (из Excel)
# На выход: word_df (одно слово = одна строка) + meta
#
# Главная метрика:
# - word_surprisal_ln_sum (sum of surprisal): сумма surprisal по токенам слова (психолингвистически корректнее)
# Доп. метрики:
# - word_surprisal_ln_mean (mean surprisal): нормировка на количество токенов
# - word_prob_equiv = exp(sum_logprob): условный эквивалент "вероятности слова"
# =========================
def build_wordlevel_for_text(text_id, title, token_df, words, soft_shift=True):
    required = {"token_id", "log_probability", "surprisal_ln", "surprisal_log2"}
    missing = required - set(token_df.columns)
    if missing:
        raise ValueError(f"token_df missing required columns: {missing}")

    token_ids = token_df["token_id"].tolist()
    pos = 0

    out_rows = []
    ok_words = 0
    nomatch_words = 0

    for wi, word in enumerate(words):
        is_first = (wi == 0)
        _, L = best_match_token_ids(token_ids, pos, word, is_first)

        if L == 0:
            nomatch_words += 1
            out_rows.append({
                "text_id": text_id,
                "title": title,
                "word_index": wi,
                "word": word,
                "tokens_in_word": None,
                "word_logprob_sum": None,
                "word_surprisal_ln_sum": None,
                "word_surprisal_log2_sum": None,
                "word_surprisal_ln_mean": None,
                "word_prob_equiv": None,
                "status": f"NO_MATCH at token_pos={pos}"
            })

            if soft_shift:
                pos += 1
                if pos >= len(token_df):
                    break
                continue
            else:
                break

        chunk = token_df.iloc[pos:pos+L]
        pos += L

        logprob_sum = float(chunk["log_probability"].sum())
        surprisal_ln_sum = float(chunk["surprisal_ln"].sum())
        surprisal_log2_sum = float(chunk["surprisal_log2"].sum())

        out_rows.append({
            "text_id": text_id,
            "title": title,
            "word_index": wi,
            "word": word,
            "tokens_in_word": int(L),
            "word_logprob_sum": logprob_sum,
            "word_surprisal_ln_sum": surprisal_ln_sum,
            "word_surprisal_log2_sum": surprisal_log2_sum,
            "word_surprisal_ln_mean": (surprisal_ln_sum / L) if L else None,
            "word_prob_equiv": math.exp(logprob_sum),
            "status": "OK"
        })
        ok_words += 1

    leftover_tokens = len(token_df) - pos

    word_df = pd.DataFrame(out_rows)
    meta = {
        "text_id": text_id,
        "title": title,
        "words_total_from_excel": len(words),
        "words_ok": ok_words,
        "words_nomatch": nomatch_words,
        "tokens_total_from_csv": len(token_df),
        "tokens_consumed": pos,
        "tokens_leftover": leftover_tokens,
        "match_rate_words": (ok_words / max(1, ok_words + nomatch_words)),
    }
    return word_df, meta


In [ ]:
# =========================
# БЛОК 5: Агрегация word-level -> text-level summary
# Делает сводку по тексту:
# - сумма/среднее surprisal по словам (OK только)
# - суммарное число токенов в словах
# - доля матчей, остаток токенов
# =========================
def wordlevel_to_textsummary(word_df, meta):
    ok = word_df[word_df["status"] == "OK"].copy()
    ok["tokens_in_word"] = ok["tokens_in_word"].astype(int)

    # если вдруг нет OK-слов
    if ok.empty:
        return {
            **meta,
            "words_ok_used": 0,
            "word_surprisal_ln_sum_text": None,
            "word_surprisal_ln_mean_text": None,
            "word_surprisal_log2_sum_text": None,
            "word_prob_equiv_text": None,
            "tokens_in_words_sum": 0,
        }

    # Главные агрегаты
    surprisal_ln_sum_text = float(ok["word_surprisal_ln_sum"].sum())
    surprisal_ln_mean_text = float(ok["word_surprisal_ln_sum"].mean())  # среднее по словам
    surprisal_log2_sum_text = float(ok["word_surprisal_log2_sum"].sum())

    # Эквивалент вероятности текста (как произведение токенов через сумму logprob):
    # exp(sum logprob по словам) = exp(sum logprob по токенам, которые вошли в слова)
    logprob_sum_text = float(ok["word_logprob_sum"].sum())
    prob_equiv_text = math.exp(logprob_sum_text)

    tokens_in_words_sum = int(ok["tokens_in_word"].sum())

    return {
        **meta,
        "words_ok_used": int(len(ok)),
        "tokens_in_words_sum": tokens_in_words_sum,
        "word_surprisal_ln_sum_text": surprisal_ln_sum_text,
        "word_surprisal_ln_mean_text": surprisal_ln_mean_text,
        "word_surprisal_log2_sum_text": surprisal_log2_sum_text,
        "word_prob_equiv_text": prob_equiv_text,
    }


In [ ]:
# =========================
# БЛОК 6: Функция "обработать выбранные тексты"
# Ты можешь вызывать её частями: например, process_texts([1,2]) потом process_texts([3,4]) и т.д.
#
# Что делает:
# - читает слова из Excel (Text_i)
# - читает token-level CSV из ZIP (Text_i_probabilities.csv)
# - строит word-level CSV
# - обновляет (дополняет) общий text-level summary CSV
# =========================
def process_texts(text_numbers):
    wb = openpyxl.load_workbook(EXCEL_PATH, data_only=True)

    # summary файл на уровне текстов
    summary_path = os.path.join(OUT_DIR, "ALL_TEXTS_WORDLEVEL_TEXTSUMMARY.csv")
    if os.path.exists(summary_path):
        summary_df = pd.read_csv(summary_path)
    else:
        summary_df = pd.DataFrame()

    new_summaries = []

    with zipfile.ZipFile(ZIP_PATH, "r") as zf:
        for i in text_numbers:
            sheet_name = f"Text_{i}"
            csv_name = f"Qwen1.5_ALL-RESULTS/Text_{i}_probabilities.csv"

            if sheet_name not in wb.sheetnames:
                print(f"[WARN] Нет листа {sheet_name} в Excel — пропускаю.")
                continue
            if csv_name not in zf.namelist():
                print(f"[WARN] Нет файла {csv_name} в ZIP — пропускаю.")
                continue

            ws = wb[sheet_name]
            words = extract_words_from_sheet(ws)

            with zf.open(csv_name) as f:
                token_df = pd.read_csv(f)

            title = token_df["title"].iloc[0] if "title" in token_df.columns and len(token_df) else None
            text_id = token_df["text_id"].iloc[0] if "text_id" in token_df.columns and len(token_df) else sheet_name

            word_df, meta = build_wordlevel_for_text(
                text_id=text_id,
                title=title,
                token_df=token_df,
                words=words,
                soft_shift=SOFT_SHIFT_ON_NO_MATCH
            )

            # сохраняем word-level CSV
            out_word_path = os.path.join(OUT_DIR, f"{sheet_name}_wordlevel.csv")
            word_df.to_csv(out_word_path, index=False, encoding="utf-8-sig")

            # делаем text-level summary
            text_summary = wordlevel_to_textsummary(word_df, meta)
            text_summary["sheet_name"] = sheet_name
            new_summaries.append(text_summary)

            print(
                f"[OK] {sheet_name} -> {out_word_path}\n"
                f"     match_rate_words={text_summary['match_rate_words']:.3f} "
                f"tokens_leftover={text_summary['tokens_leftover']} "
                f"words_ok={text_summary['words_ok']}/{text_summary['words_total_from_excel']}"
            )

    if not new_summaries:
        print("[INFO] Ничего не обработано.")
        return

    new_df = pd.DataFrame(new_summaries)

    # обновляем summary: если текст уже был — заменяем строку; иначе добавляем
    if summary_df.empty:
        summary_df = new_df
    else:
        # ключ для уникальности
        key_cols = ["sheet_name"]
        summary_df = summary_df[~summary_df["sheet_name"].isin(new_df["sheet_name"])]
        summary_df = pd.concat([summary_df, new_df], ignore_index=True)

    # сортировка по номеру текста (если возможно)
    def _extract_num(s):
        try:
            return int(str(s).split("_")[1])
        except:
            return 10**9

    summary_df["text_num"] = summary_df["sheet_name"].apply(_extract_num)
    summary_df = summary_df.sort_values("text_num").drop(columns=["text_num"])

    summary_df.to_csv(summary_path, index=False, encoding="utf-8-sig")
    print(f"\n[SUMMARY UPDATED] {summary_path}")
    display(summary_df)


In [ ]:
# =========================
# БЛОК 7: Запуск примера (постепенная обработка)
# Меняй список как хочешь: [1,2] затем [3,4] ...
# =========================
process_texts([1, 2])
